In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn import model_selection
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.svm import SVR
from sklearn import metrics
import random
import yaml
import joblib

In [2]:
f = open("params.yaml", "r")
params = yaml.load(f, Loader=yaml.SafeLoader)
f.close()

In [3]:
def read_data(params):
    x_train = joblib.load(params['DUMP_TRAIN'])
    y_train = joblib.load(params['Y_PATH_TRAIN'])
    x_valid = joblib.load(params['DUMP_VALID'])
    y_valid = joblib.load(params['Y_PATH_VALID'])

    return x_train, y_train, x_valid, y_valid

In [190]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn import model_selection
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.svm import LinearSVR
from sklearn import metrics
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
import yaml
from xgboost import XGBRegressor

x_train, y_train, x_valid, y_valid  = read_data(params)

def model_search(x_train, y_train):
    ts_cv = TimeSeriesSplit(
        n_splits=5,
        max_train_size=None,
    )

    models = []
    models.append(('RandomForrest', RandomForestRegressor()))
    models.append(('Lasso', Lasso()))
    models.append(('Ridge', Ridge()))
    models.append(('SVR', LinearSVR()))
    
    results = []
    names = []
    scoring = 'neg_mean_absolute_error'
    for name, model in models:
        #kfold = model_selection.KFold(n_splits=5, shuffle=False)
        cv_results = model_selection.cross_val_score(model, x_train, y_train, cv=ts_cv, scoring=scoring)
        results.append(cv_results)
        names.append(name)
    results_mean = [sum(x)/5 for x in results]
    df = pd.DataFrame(list(zip(names, results_mean)), columns =['model', 'score'])
    df = df.set_index('model')
    mode_name = df.idxmax()
    mkd = mode_name[0]
    return mkd

In [5]:
def normalization(x_all,
                  state = 'fit'):
    index = x_all.index
    cols = x_all.columns
    

    if state == 'fit':
        normalizer = StandardScaler()
        normalizer.fit(x_all)
        joblib.dump(normalizer,
                    "output/normalizer.pkl")

    elif state == 'transform':
        normalizer = joblib.load("output/normalizer.pkl")
        
    normalized = normalizer.transform(x_all)
    normalized = pd.DataFrame(normalized)
    normalized.index = index
    normalized.columns = cols
    return normalized

In [6]:
x_train_normalised = normalization(x_train)

In [191]:
yut = model_search(x_train_normalised, y_train)
yut

'Ridge'